# Prediction check using trained model

This notebook eavluate result of train_virtual_categories_138_1_vs_all.ipynb.

## 1. Set up

In [ ]:
import os
import json
import glob
import scipy
import numpy as np

# import preprocess func from processor module
from processor import preprocess
from processor import DataSet

from keras.models import model_from_json
from keras.models import model_from_yaml
from keras.utils import np_utils
from keras.preprocessing import image

import pandas as pd
import glob

In [ ]:
# set root as ../
import sys
sys.path.append("../")

In [ ]:
DATA_DIR = ""

BASE_DATA_DIR= os.path.normpath(os.path.join(DATA_DIR, ''))

# created by virtual_categories.ipynb
CATSTORE_PATH = ""
CATEGORY_DICT_CSV=os.path.join(BASE_DATA_DIR, "")

In [ ]:
BASE_MODEL_DIR = "./trained_model/inceptionv3/"
BASE_NAME = ""
CODE_EXTRACTOR=""
BASE_CODE_NAME = "code_" + BASE_NAME

In [ ]:
from one_vs_all import ModelBinder
from codeextractor import DNNCodeExtractor

def ModelBinder_create(base_model_name = BASE_CODE_NAME, basedir = BASE_MODEL_DIR, extractor_path=CODE_EXTRACTOR):
    return ModelBinder(base_model_name, basedir, DNNCodeExtractor.create_from(extractor_path))

In [ ]:
CHECK_DATA_DIR = ''

In [ ]:
CHECK_DATA_PATHS = list(glob.glob(os.path.normpath("{}/*.jpg").format(CHECK_DATA_DIR)))

In [ ]:
len(CHECK_DATA_PATHS)

# Ensemble predict

In [ ]:
ens = ModelBinder_create()

In [ ]:
from category import VirtualCategories
import os

In [ ]:
vc = VirtualCategories.from_file(CATSTORE_PATH)

In [ ]:
catkeys = vc.keys()

In [ ]:
modelkeys = catkeys

In [ ]:
%%time

ens.load_all_models(modelkeys)

In [ ]:
df = ens.predict(CHECK_DATA_PATHS)

In [ ]:
ens.add_argmax_max(df)

In [ ]:
df.to_pickle("")

In [ ]:
# for read
df = pd.read_pickle("")

In [ ]:
df_sorted = df.sort_values(by="max", ascending= False)

In [ ]:
df_sorted[df_sorted.columns[-5:]][0:10]

# Argmax version, evaluation

In [ ]:
from visualize import plot_image_list

In [ ]:
def plot_target_top10(target_key):
    maxesdf = df_sorted.where(df_sorted["argmax"] == target_key).dropna()
    print(vc.name(target_key))
    maxes = maxesdf[0:10]["max"].values
    print(", ".join(str(val) for val in maxes[0:5]))
    print(", ".join(str(val) for val in maxes[5:]))
    plot_image_list(maxesdf[0:10]['filepaths'].values)

In [ ]:
# plot_target_top10('51')

In [ ]:
# plot_target_top10('165')

In [ ]:
def plot_target_top20(target_key):
    maxesdf = df_sorted.where(df_sorted["argmax"] == target_key).dropna()
    print(vc.name(target_key))
    maxes = maxesdf[0:20]["max"].values
    print(", ".join(str(val) for val in maxes[0:5]))
    print(", ".join(str(val) for val in maxes[5:10]))
    print(", ".join(str(val) for val in maxes[10:15]))
    print(", ".join(str(val) for val in maxes[15:]))
    plot_image_list(maxesdf[0:20]['filepaths'].values)

In [ ]:
vc.name('0')

In [ ]:
# plot_target_top20('0_128')

In [ ]:
# plot_target_top10('0_128')

In [ ]:
# plot_target_top10('0_128')

In [ ]:
# plot_target_top10('11_13_54_120_130_58_86_96')

In [ ]:
# plot_target_top10('164')

In [ ]:
# plot_target_top10('100')

In [ ]:
# plot_target_top10('136_24')

In [ ]:
# plot_target_top10('136_24')

In [ ]:
# plot_target_top10('101')

In [ ]:
max0_128 = df_sorted.where(df_sorted["argmax"] == "0_128").dropna()

In [ ]:
eggs = df_sorted.where(df_sorted["argmax"] == "136_24").dropna()[0:10]

In [ ]:
eggs

In [ ]:
eggs.iloc[0, :-3].sort_values()

In [ ]:
eggs.iloc[6:10, -3]

In [ ]:
eggs.iloc[6, -3]

In [ ]:
from IPython.display import Image, display

In [ ]:
eggs.iloc[6, :-3].sort_values()

In [ ]:
handled = set()
sorted_cats = []
for ag in df_sorted["argmax"].values:
    if ag not in handled:
        handled.add(ag)
        sorted_cats.append(ag)
        

In [ ]:
sorted_cats

In [ ]:
# after 10epoch x 2 retraining
# plot_target_top10("1_144")

In [ ]:
# before
# plot_target_top10("1_144")

In [ ]:
# plot_target_top10('56')

In [ ]:
# plot_target_top10('141_157')

In [ ]:
# plot_target_top10('140')

In [ ]:
# plot_target_top10('137_170')

In [ ]:
# plot_target_top10("10")

In [ ]:
# plot_target_top10("16_32_87")

In [ ]:
# plot_target_top10('63')

In [ ]:
# plot_target_top10("65")

In [ ]:
# plot_target_top10("112")

In [ ]:
# plot_target_top10("102")

In [ ]:
# plot_target_top10("108_72")

In [ ]:
import pickle
with open("tmp/accs.dat", "rb") as f:
    accs_list = pickle.load(f)

In [ ]:
accsdict = {tup[0]:tup[1][1] for tup in zip(modelkeys, accs_list)}

In [ ]:
accsdict

In [ ]:
vc.namesdict

In [ ]:
vc.keys().index("25_48")

In [ ]:
df_sorted[-50:]["max"]

# Try and error

In [ ]:
checkbtlnks = ens.files2bottlenecks(CHECK_DATA_PATHS)

In [ ]:
checkbtlnks.shape

In [ ]:
models = ens._models

In [ ]:
models['0_128'].predict(checkbtlnks)

In [ ]:
list(models.keys())[0:3]

In [ ]:
preddict = {key: models[key].predict(checkbtlnks)[:, 1] for key in models.keys() }

In [ ]:
catkeys = models.keys()

In [ ]:
preddict["filepaths"] = CHECK_DATA_PATHS

In [ ]:
df = pd.DataFrame(preddict)

In [ ]:
len(df.columns[:-2])

In [ ]:
df.columns[0:len(catkeys)]

In [ ]:
df['argmax'] = df[df.columns[:-1]].idxmax(axis=1)

In [ ]:
df['max'] = df[df.columns[0:len(catkeys)]].max(axis=1)

In [ ]:
from visualize import plot_image_list

In [ ]:
max101 = df.where(df["argmax"] == "101").dropna().sort_values(by="101", ascending=False)

In [ ]:
max101[0:5]['max']

In [ ]:
# plot_image_list(max101[0:10]['filepaths'].values)

In [ ]:
max101 = df.where(df["argmax"] == "101").dropna().sort_values(by="101", ascending=False)

In [ ]:
sys.modules['visualize']

In [ ]:
del sys.modules['visualize']

In [ ]:
res0_128 = Out[53]

In [ ]:
res0_128.shape

In [ ]:
res113 = models['113'].predict(checkbtlnks)

In [ ]:
res0_128[:, 1]

In [ ]:
from codeextractor import DNNCodeExtractor

In [ ]:
ens.extractor = DNNCodeExtractor.create_from(CODE_EXTRACTOR)

In [ ]:
from importlib import reload

In [ ]:
reload(sys.modules["codeextractor"])

In [ ]:
import sys
sys.modules["codeextractor"]

In [ ]:
del sys.modules["codeextractor"]

In [ ]:
dir(DNNCodeExtractor)

In [ ]:
from codeextractor import fullmodel_file_to_codemodel_file

In [ ]:
topath = [os.path.join(BASE_MODEL_DIR, "code_{}_{}".format(BASE_NAME, key)) for key in modelkeys]

In [ ]:
frompath = [os.path.join(BASE_MODEL_DIR, "{}_{}".format(BASE_NAME, key)) for key in modelkeys]

In [ ]:
def full2code(fromp, to):
    print("convert {}=>{}".format(fromp, to))
    fullmodel_file_to_codemodel_file(fromp, to)

In [ ]:
[full2code(pair[0], pair[1]) for pair in zip(frompath, topath)]

In [ ]:
[full2code(pair[0], pair[1]) for pair in zip(frompath[2:], topath[2:])]

In [ ]:
target_key= '1_144'

In [ ]:
model_save_path = os.path.join(BASE_MODEL_DIR, "{}_{}".format(BASE_NAME, target_key))

In [ ]:
model_save_path

In [ ]:
from modelutils import load_model

In [ ]:
model = load_model(model_save_path)

In [ ]:
vc.name(target_key)

In [ ]:
test_files_dict = vc.list_dict(os.path.join(BASE_DATA_DIR, "valid") )

In [ ]:
from category import split_files
from processor import DataSet

In [ ]:
truetests, falsetests = split_files(target_key, test_files_dict)

In [ ]:
ds = DataSet()

In [ ]:
truearrs = ds.files_to_dataset(truetests)

In [ ]:
pred_trues = model.predict(truearrs)

In [ ]:
pred_trues.shape

In [ ]:
truearrs = None

In [ ]:
chunks = ds.chunked(falsetests, 3000)

In [ ]:
pred_list = []
for chunk in chunks:
    arr = ds.files_to_dataset(chunk)
    prediction = model.predict(arr)
    pred_list.append(prediction)

In [ ]:
len(pred_list)

In [ ]:
len(falsetests)

In [ ]:
pred_list[0].shape

In [ ]:
pred_col = []
for el in pred_list:
    pred_col.extend(el)

In [ ]:
len(pred_col)

In [ ]:
pred_col[0].shape

In [ ]:
len(pred_trues)

In [ ]:
pred_trues_list = [tup for tup in pred_trues]

In [ ]:
len(pred_trues_list)

In [ ]:
predcat = pred_trues_list+pred_col

In [ ]:
from itertools import repeat

In [ ]:
list(repeat(0, 10))+list(repeat(1, 5))

In [ ]:
RESULT_ONEVS = pd.DataFrame({
    'ans' : list(repeat(1, len(truetests))) + list(repeat(0, len(falsetests))),
    'class0': [pair[0] for pair in predcat],
    'class1' : [pair[1] for pair in predcat],
    'filepaths' : truetests+falsetests
})

In [ ]:
labels = RESULT_ONEVS.where(RESULT_ONEVS['ans'] == 1).dropna()

In [ ]:
df = labels.sort_values(by="class1")

In [ ]:
df[0:5]

In [ ]:
# plot_image_list(df[0:10]["filepaths"].values)

In [ ]:
highconf = RESULT_ONEVS.sort_values(by="class1", ascending=False)

In [ ]:
# plot_image_list(highconf[0:10]['filepaths'].values)